### Importing data

In [2]:
import json
import os

def import_data(filepath): 
  """
  Imports data from a JSON file and converts it to a string format.

  Parameters:
  filepath (str): Path to the JSON file.

  Returns:
  str: Data from the JSON file as a single string with elements separated by newlines.

  Raises:
  FileNotFoundError: If the file does not exist.
  ValueError: If the file content is not a valid JSON.
  TypeError: If the JSON content is not a list of strings.
  """
  if not os.path.exists(filepath):
      raise FileNotFoundError(f"The file '{filepath}' does not exist.")
  
  try:
      with open(filepath, 'r') as file:
          data = json.load(file)
      if not isinstance(data, list) or not all(isinstance(item, str) for item in data):
          raise TypeError("The JSON content must be a list of strings.")
      return "\n".join(data)
  
  except json.JSONDecodeError as e:
      raise ValueError(f"Failed to decode JSON: {e}")

### Prompt 

In [4]:
from openai import OpenAI

prompt_summarize = """
Jesteś specjalistą od analizy danych medycznych. Przeczytaj podane raporty (konsultacje) szpitalne i przyporządkuj kazdy do jednej z dwóch kategorii:
1. **Kategoria 1**: Konsultacja opisuje aktywną resuscytację pacjenta, obejmującą działania takie jak: podjęcie resuscytacji, intubacja, podanie leków ratunkowych (np. adrenaliny), użycie defibrylatora, wykonywanie masażu serca itp.
2. **Kategoria 2**: Konsultacja odnosi się do wcześniejszej resuscytacji, ale nie opisuje aktywnej resuscytacji w trakcie danej konsultacji (np. stan pacjenta po wcześniejszym NZK, informacje o przebytej resuscytacji bez szczegółów interwencji w obecnym czasie).

Twoje zadanie:
1. Jeśli raport należy do **Kategorii 1**, zamień raport na zestawienie w formie słów kluczowych. Słowa kluczowe mają być zapisane w mianowniku i w pełni standaryzowane. Oto słowa kluczowe, z których masz korzystać w odpowiedzi i z niczego innego:
   - **Mechanizmy**: "asystolia" (bradyasystolię tez traktuj jako asystolię), "VF" (migotanie komór), "PEA" (aktywność elektryczna bez tętna).
   - **Czas**: czas trwania resuscytacji w minutach, określony jako różnica czasu zakończenia resuscytacji i jej rozpoczęcia lub wystąpienia NZK ("czas X"). Jeśli brak informacji, pomiń parametr czasu.
   - **Procedury**: "intubacja", "lucas", "ambu", "masaż serca", "defibrylacja", podane leki: "adrenalina", "bikarbonat", "amiodaron", "inne leki".
   - **Wynik**: "ROSC", "zgon".

   Jeśli w raporcie brakuje danych, pomiń te parametry w odpowiedzi. Zachowaj standardowy format wyjściowy.

2. Jeśli raport należy do **Kategorii 2**, zwróć dokładnie tekst: 'BRAK'.

### Przykłady
**Raporty:**  
["Wezwanie na interwencję z powodu nagłego zatrzymania krążenia (NZK). Według relacji personelu SOR, zatrzymanie krążenia nastąpiło około godziny 14:30 w mechanizmie asystolii. Natychmiast rozpoczęto RK, wykonując masaż zewnętrzny serca oraz intubując pacjenta przez usta. Rozpoczęto wentylację przy pomocy worka Ambu, a masaż serca kontynuowano przy użyciu urządzenia Lucas.
W trakcie działań podano łącznie 6 mg adrenaliny. Pomimo przeprowadzonych działań resuscytacyjnych, nie uzyskano ROSC. O godzinie 15:00 stwierdzono zgon pacjenta.", "Przy przybyciu pacjent był masowany oraz wentylowany za pomocą worka Ambu. Pacjenta zaintubowano przez usta rurką 8.5. Podawano adrenalinę zgodnie z aktualnymi wytycznymi. Pomimo prowadzonej akcji resuscytacyjnej nie uzyskano ROSC. O godzinie 16:10 stwierdzono zgon chorego.", "Pacjent NN o nieznanej przeszłości chorobowej, zaniedbany higienicznie. Stan po NZK i skutecznej resuscytacji wykonanej przez ZRM. Obecnie pacjent w stanie śpiączki, wentylowany mechanicznie, hemodynamicznie stabilny na wlewie noradrenaliny. W wykonanym TK głowy wykluczono krwawienie. Zalecenia: pełne monitorowanie, kontrolne badania laboratoryjne, RTG klatki piersiowej, ewentualna decyzja o koronarografii po ocenie stanu neurologicznego."]
**Odpowiedź:**  
["asystolia, masaż serca, intubacja, ambu, lucas, adrenalina, czas 30, zgon", "masaż serca, intubacja, ambu, adrenalina, zgon", "BRAK"]
"""

def get_response(prompt):
    """
    Sends a prompt to an external chat model (gpt-4-turbo) and retrieves the response.

    Parameters:
    prompt (str): The input string to be processed by the model.

    Returns:
    str: The response generated by the model.

    Raises:
    ValueError: If the prompt is empty or invalid.
    Exception: For any unexpected errors during the API call.
    """
    try:
        if not isinstance(prompt, str) or not prompt.strip():
            raise ValueError("Prompt must be a non-empty string.")
        
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            temperature=0.0,
            messages=[
                {"role": "system", "content": "Jesteś pomocnym asystentem."},
                {"role": "user", "content": prompt}
            ],
        )
        
        assistant_response = response.choices[0].message.content
        
        if not assistant_response or not isinstance(assistant_response, str):
            raise ValueError("Model response is empty or invalid.")
        
        return assistant_response.strip()

    except ValueError as ve:
        print(f"Input or response validation error: {ve}")
        raise  
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise  

def summarize_batches(batches):
    """
    Processes batches of medical reports and combines their summarized results.

    Parameters:
    batches (list of list of str): A list of batches, where each batch is a list of medical reports.

    Returns:
    list of str: Combined list of summarized reports without leading numbers or extra quotes.
    """
    combined_summaries = []

    for batch in batches:
        try:
            batch_content = "\n\n".join(batch)
            final_prompt = prompt_summarize + f"\n\n<raporty>\n{batch_content}\n</raporty>"
            
            response = get_response(final_prompt)
            
            summaries = [item.strip() for item in response.split("\n") if item.strip()]
            
            clean_summaries = []
            for summary in summaries:
                clean_summary = summary.split(".", 1)[-1].strip()
                clean_summary = clean_summary.strip('"')
                clean_summaries.append(clean_summary)
            
            combined_summaries.extend(clean_summaries)
        
        except Exception as e:
            print(f"Error processing batch: {batch}\nError: {e}")

    return combined_summaries

In [40]:
import tiktoken

def split_reports_into_batches(report_list, prompt, max_tokens=4096):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    prompt_tokens = len(tokenizer.encode(prompt))
    max_tokens_per_batch = max_tokens - prompt_tokens

    batches = []
    current_batch = []
    current_tokens = 0

    for report in report_list:
        report_tokens = tokenizer.encode(report)
        report_token_count = len(report_tokens)

        if current_tokens + report_token_count > max_tokens_per_batch:
            batches.append(current_batch)
            current_batch = []
            current_tokens = 0

        current_batch.append(report)
        current_tokens += report_token_count

    if current_batch:
        batches.append(current_batch)

    return batches

In [74]:
def write_file(str_data, fname="summ_data.txt"):
    """
    Checks if a specified file exists. If the file does not exist, it generates summarized data 
    using the provided summarize_func and writes it to the file.

    Parameters:
    fname (str): The name of the file to check or create, default is "summ_data.txt".

    Returns:
    None
    """
    try:
        if os.path.isfile(fname):
            print(f"File '{fname}' already exists. No action taken.")
            return

        with open(fname, "w") as f:
            f.write(str_data)

    except FileNotFoundError as fnf_error:
        print(f"File not found error: {fnf_error}")
        raise  

    except ValueError as ve:
        print(f"Value error: {ve}")
        raise

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [70]:
filepath = "/Users/jk620/Desktop/Praca_inżynierska/konsultacje.json"

data = import_data(filepath)
data = data.split("\n")
batches = split_reports_into_batches(data, prompt_summarize)

summaries = summarize_batches(batches)
print(summaries)


['asystolia, masaż serca, intubacja, ambu, lucas, adrenalina, czas 30, zgon', 'BRAK', 'asystolia, masaż serca, intubacja, ambu, adrenalina, zgon', 'VF, masaż serca, intubacja, adrenalina, amiodaron, defibrylacja, inne leki, ROSC', 'asystolia, masaż serca, intubacja, ambu, adrenalina, zgon', 'BRAK', 'asystolia, masaż serca, intubacja, bikarbonat, adrenalina, inne leki, czas 8, ROSC', 'asystolia, masaż serca, intubacja, ROSC', 'BRAK', 'VF, masaż serca, intubacja, defibrylacja, adrenalina, ROSC', 'BRAK', 'VF, masaż serca, intubacja, defibrylacja, adrenalina, zgon', 'asystolia, masaż serca, intubacja, adrenalina, ROSC', 'asystolia, masaż serca, intubacja, lucas, adrenalina, czas 10, ROSC, inne leki', 'VF, asystolia, masaż serca, intubacja, adrenalina, zgon', 'asystolia, masaż serca, intubacja, ambu, inne leki, zgon', 'asystolia, masaż serca, intubacja, ambu, adrenalina, bikarbonat, czas 25, ROSC', 'asystolia, masaż serca, intubacja, inne leki, zgon', 'asystolia, masaż serca, intubacja, luc

In [6]:
key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=key)
print(key)

# pozbyc sie BRAK
# sprawdzic stosunek BRAK do wszystkich 
# dac dane do pandas? 

sk-proj-upFwAYD23HiRbUiSjiMUT3BlbkFJarFyI4evt8sILhomXDpd
